## Gemini Pro 에서 Vertex AI Search를 Grounding Service 로 사용하는 방법.

Feedback : shins777@gmail.com. 

* 이 Colab은 Gemini Pro 의 기본 동작을 설명하기 위한 예제입니다.  


### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [3]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [8]:
model_name="gemini-1.0-pro"
project="ai-hangsik"
location="asia-northeast3"

In [6]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Tool, grounding

from IPython.display import display, Markdown

#### 초기화 및 모델 생성

In [9]:
vertexai.init(project=project, location=location)
model = GenerativeModel(model_name)

In [10]:
PROMPT = (
    "특별법에서 민간 부문의 양자정보통신기술개발 지원에 대해서 상세하게 설명해주세요."
)

#### Foundation model 내의 데이터를 기반으로 생성된 정보

In [11]:
response = model.generate_content(PROMPT)
display(Markdown(response.candidates[0].text))

**민간 부문 양자정보통신기술개발 지원**

양자정보통신특별법은 민간 부문의 양자정보통신기술(QICT) 개발을 지원하기 위해 다음과 같은 조치를 규정하고 있습니다.

**1. 연구개발 지원:**

* 정부는 민간기업의 QICT 연구개발(R&D) 프로젝트에 지원금과 세제 지원을 제공합니다.
* 지원 금액과 기간은 프로젝트 규모와 기술적 중요성에 따라 달라집니다.
* 연구개발 지원은 기초 연구부터 응용 연구와 개발까지 모든 연구 단계를 포함합니다.

**2. 인프라 구축 지원:**

* 정부는 양자 네트워크, 양자 컴퓨터 및 기타 관련 인프라 구축을 위한 자금을 제공합니다.
* 인프라는 산업 연구, 상용 서비스 개발 및 시범 프로젝트에 사용될 수 있습니다.

**3. 인재 육성 지원:**

* 정부는 QICT 전문가의 교육과 훈련을 위한 프로그램에 자금을 제공합니다.
* 프로그램에는 대학 및 연구 기관의 학술 연구 지원, 산업 인력 훈련 및 해외 유학 지원이 포함됩니다.

**4. 규제 지원:**

* 정부는 QICT 개발을 위한 유리한 규제 환경을 조성합니다.
* 이러한 조치는 양자 기술 개발 및 상용화에 필요한 허가와 인증 절차의 간소화를 포함합니다.
* 정부는 또한 데이터 보안 및 개인 정보 보호 관련 문제를 해결하는 데 도움을 주는 규제 프레임워크를 제공합니다.

**5. 국제 협력:**

* 정부는 다른 국가 및 국제 기관과 협력하여 QICT 개발을 촉진합니다.
* 이러한 협력에는 기술 공유, 공동 연구 프로젝트 및 국제 표준화 노력이 포함됩니다.

**양자정보통신특별법은 다음을 포함한 특정 산업 분야에 중점을 둡니다:**

* 양자 암호화 및 양자 네트워킹
* 양자 컴퓨팅 및 양자 시뮬레이션
* 양자 측정 및 검출
* 양자 소재 및 양자 소자
* 양자 소프트웨어 및 알고리즘

**추가 지원 조치**

특별법 이외에도 정부는 민간 부문의 QICT 개발을 지원하는 데 도움이 되는 추가 조치를 시행하고 있습니다.

* **과학기술정보통신부(MSIT) 양자정보통신산업화본부:** QICT 산업 생태계 구축 및 촉진을 담당하는 기관입니다.
* **양자정보통신산업연구원:** 양자 기술의 연구, 개발 및 상용화를 지원하는 공공연구기관입니다.
* **양자정보통신산업협회:** QICT 기업과 연구 기관을 위한 산업 단체입니다.

이러한 지원 조치를 통해 정부는 대한민국이 세계적인 QICT 혁신 허브가 되도록 돕는 것을 목표로 합니다.

In [12]:
DATA_STORE_PROJECT_ID = "ai-hangsik"
DATA_STORE_REGION = "global"
DATA_STORE_ID = "hackathon-ds_1709838247983"

#### Vertex AI Search 내의 데이터를 검색해서 생성된 정보

In [13]:
datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"
tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=datastore))
)

response = model.generate_content(PROMPT, tools=[tool])

display(Markdown(response.candidates[0].text))

**민간 부문의 양자정보통신 기술 개발 지원**

정부는 민간 부문의 양자정보통신 기술 개발을 지원하기 위해 다음과 같은 지원을 제공할 수 있습니다.

* 행정적 및 기술적 지원
* 기술 집약적 중소기업 및 벤처기업 지원
* 양자정보통신 기술을 사용하여 창업하는 기업 지원